In [6]:
import numpy as np 
import pandas as pd 

Open Stock price prediction using previous dates stock details, like open, high, low and volume.
https://www.kaggle.com/medharawat/google-stock-price

In [7]:
df_train = pd.read_csv('/kaggle/input/gooogle-stock-price/Google_Stock_Price_Train.csv')
df_test = pd.read_csv('/kaggle/input/gooogle-stock-price/Google_Stock_Price_Test.csv')

In [8]:
df_train.dtypes

In [9]:
df_test.dtypes

In [10]:
df_train['Close'] = df_train['Close'].str.replace(',', "")
df_train['Volume'] = df_train['Volume'].str.replace(',', "")
df_test['Volume'] = df_test['Volume'].str.replace(',', "")

In [11]:
df_train.dtypes

In [12]:
df_train['Close'] = pd.to_numeric(df_train['Close'])
df_train['Volume'] = pd.to_numeric(df_train['Volume'])
df_test['Volume'] = pd.to_numeric(df_test['Volume'])

In [13]:
df_train.dtypes

In [14]:
df_test.dtypes

In [15]:
import matplotlib.pyplot as plt

In [16]:
plt.plot(pd.to_datetime(df_train['Date']),df_train['Open'])

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_train[['Open']])
XX_scaled = scaler.fit_transform(df_train[['Open', 'High', 'Low', 'Close', 'Volume']])


In [18]:
from sklearn.preprocessing import MinMaxScaler
sca1 = MinMaxScaler()
XX_test_scaled_open = sca1.fit_transform(df_test[['Open']])

In [19]:
XX_test_scaled = scaler.fit_transform(df_test[['Open', 'High', 'Low', 'Close', 'Volume']])


In [20]:
X_train = []
for i in range(1198):
    X_train.append(X_scaled[i:i+61, 0])
    
X_train = np.array(X_train)
X_train.shape

In [21]:
XX = []
for j in range(5):
    YY = []
    for i in range(1198):
        YY.append(XX_scaled[i:i+61, j])
    XX.append(YY)
XX = np.array(XX)
XX.shape

In [22]:
XX_train = XX[:,:,:60]


In [23]:
#X_train , y_train = X[:,:60], X[:,-1]
#X_train = X[:,:60]
y_train = X_train[:,-1]
#X_train.shape, y_train.shape
XX_train.shape, y_train.shape

In [24]:
XX_train = XX_train.reshape(1198,60,5)

In [25]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

In [26]:
model = Sequential()
model.add(LSTM(100, return_sequences = True, input_shape = (60,5)))
model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100, return_sequences = False))
model.add(Dense(1))

In [27]:
model.compile(loss = 'mean_squared_error')

In [28]:
hist = model.fit(XX_train,y_train, batch_size=32, epochs=50)

In [29]:
plt.plot(hist.history['loss'][1:])

In [30]:
XX_test = np.concatenate([XX_scaled[-60:], XX_test_scaled], axis=0)

In [31]:
XX_test.shape

In [32]:
X = []
for j in range(5):
    YY = []
    for i in range(20):
        YY.append(XX_test[i:i+60, j])
    X.append(YY)
X = np.array(X)
X.shape

In [33]:
X = X.reshape(20,60,5)

In [34]:
y_pre = model.predict(X)
y_pre

In [35]:
df_test['Open']

In [36]:
y=sca1.inverse_transform(y_pre)
y

In [37]:
plt.plot(df_test['Open'], c='blue')
plt.plot(y, c='red')